<a href="https://colab.research.google.com/github/gucher/Remote_Grave_Digger/blob/master/Remote_Grave_Digger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How I built a remote controlled Grave Digger with a Raspberry Pi

*This is part 1 of a series in which I take a typical RC car and add numerous technology and software upgrades in pursuit of a fully autonomous Grave Digger.*

*In Part 1 we will explore:*

*   Deconstruction of RC car
*   Default wiring and framework
*   Battery source reconstruction
*   Wiring and electircal setup
*   Python libraries
*   Motor and speed setting (PWM)
*   Using curses library

## Original Car

#### At first I was looking to find a smaller car with a wide range of motion, but I laid eyes on this beauty at Big Lots and couldn't resist the sale price of $20.

![alt text](https://drive.google.com/uc?id=13omzro0UhENkenbvYvJRBl4YPV9l8G-u)

## Deconstruction

Removing the body, we see a plastic frame and two sets of wiring to the front and rear axels.

![alt text](https://drive.google.com/uc?id=1Jt_gmCn6PJC8Fm7YF5gisILsQaJpuvX4)

The bottom of the car holds the battery pack and power on/off switch. Front and rear axels are removable.

![alt text](https://drive.google.com/uc?id=1h5xrLZJFlOmDnFv1xUKZjXejvJf3xGiU)

Default controller hooked up to battery pack.

![alt text](https://drive.google.com/uc?id=12mlGh7Xx7VV-Azu4HwA2Kw8vAb2Jw79m)

Remove all components of RC car except front and rear DC motors.

![alt text](https://drive.google.com/uc?id=19-H_gTRc5r_uFdRB3mRfajZYaxSFyaWi)

*I explored the idea of switching out tires but decided against it as the project went on.*

Here's the RC car broken down with all standard peices. I was surprised with how simple these cheap RC cars are. They consist of very limited and basic components.

![alt text](https://drive.google.com/uc?id=1ZXl4hqj67dI5kv8Kw42NUoPqW9bTbX2c)

To determine which GPIO pins control F/B and L/R control, I used an extra DC motor for testing.

![alt text](https://drive.google.com/uc?id=1Q-495oJAm4tE2RNQASTLMjkM5kY-43Y3)

*I am not an electonics expert (this is my first real project), so I relied on tutorials and infographics a lot.*

Setup for controlling a DC motor with L298N.

![alt text](https://drive.google.com/uc?id=1ViGeNgD4M08J_eAbHheEs4iY6LE7O7e2)

To lower the speed at which the rear motor cycles, I used PWM (Pulse Width Modulation) capable GPIO pins.

![alt text](https://drive.google.com/uc?id=1aooWqVkVjGv4NIorVhxvBA799YnOCFoi)

Once the L298N was wired to front and rear motors, I affixed a rectangle of cardboard to the bottom as a placeholder to rest all the components on.

![alt text](https://drive.google.com/uc?id=1dle7K6xfJX2IS-xWxRvbaYDILRfbRPR8)

Here is the car with motor setup and battery power.

![alt text](https://drive.google.com/uc?id=1Jz27PFRd7CLC524b6OPJrqiqvlieS1b_)

![alt text](https://drive.google.com/uc?id=1LQsxerijTHog5nP7tkvWK1vBc0DmJVCp)

*I connected a simple On/Off switch and placed the battery pack at the base. Later in the project I switched out the battery pack pictured above and drew power straight from the L298N, which was powered by a separate 8AA pack.*

For testing I neglected any cable (wire) management and opted to squeeze everything under the FLIMSY plastic frame. Here's how it turned out without the shell attached (resting in background).

![alt text](https://drive.google.com/uc?id=1Hfd_o7ZjdhfZmxTPOz9fdoIuhWI5cnQs)

## Now let's look at some code...

To run all code I booted up a Remote Desktop and programmed locally on the RPi. There are numerous tutorials online to setup RPi RDC and PuTTY.

In [0]:
# import libraries

import RPi.GPIO as GPIO
import time
import curses

In [0]:
# I am using BOARD setup here. For BCM setup GPIO pins will need updated in below code

GPIO.setmode(GPIO.BOARD)

In [0]:
# PWM pins 12, 32, 33, 35

GPIO.setup(33,GPIO.OUT, initial=0) # <-- initial set to 0
GPIO.setup(35,GPIO.OUT, initial=0) # <-- to ensure motors will not run when RPi is turned on
GPIO.setup(13,GPIO.OUT, initial=0) # <-- depending which pins are used
GPIO.setup(15,GPIO.OUT, initial=0) # <-- initializing may not be needed

In [0]:
# setup curses
# to remove text from displaying in terminal when controlling car direction and movement, use noecho

screen = curses.initscr()
# curses.noecho() <-- remove comment depending on preference above

curses.cbreak()
curses.keypad(True)

In [0]:
# create variables for front and backwards control

f = GPIO.PWM(33,100) # <-- note the PWM capable pins from above
b = GPIO.PWM(35,100) # <-- set 100 for initial frequency

Quick reference on using PWM in RPi.GPIO

https://sourceforge.net/p/raspberry-gpio-python/wiki/PWM/

In [0]:
# start your engines!

f.start(0) # starting at 0% duty cycle
b.start(0)

In [0]:
# it's a good idea to include a code break and print reminder keys (in addition to stopping the script)

print('Press th [q] key to quit.')
print('Press the [b] key to stop the car.')
print('Press [Ctrl + c] to stop the script.')
time.sleep(5) # <-- for us slow readers

### We'll be using try / finally for exception handling. 

*The try statement has another optional clause which is intended to define clean-up actions that must be executed under all circumstances.*

*A finally clause is always executed before leaving the try statement, whether an exception has occurred or not. When an exception has occurred in the try clause and has not been handled by an except clause (or it has occurred in an except or else clause), it is re-raised after the finally clause has been executed. The finally clause is also executed “on the way out” when any other clause of the try statement is left via a break, continue or return statement.*

Reference the documentation for additional information: https://docs.python.org/3/tutorial/errors.html

In [0]:
try:
  while True: # <-- loop will run until broken
    char == screen.getch()
    if char == ord('q'):
      break # code will break and move to finally
    elif char == curses.KEY_DOWN:
#     to adjust the value of duty cycle, use ChangeDutyCycle
#     the value can be between 0% (LOW) and 100% (HIGH)
      b.ChangeDutyCycle(30)
      f.ChangeDutyCycle(0)
#     although f is already set to 0%
#     it is good practice to control DC of each variable when changing
    elif char == curses.KEY_UP:
      b.ChangeDutyCycle(0)
      f.ChangeDutyCycle(30)
    elif char == curses.KEY_LEFT:
      GPIO.output(13,GPIO.HIGH)
      GPIO.output(15,GPIO.LOW)
    elif char == curses.KEY_RIGHT:
      GPIO.output(13,GPIO.LOW)
      GPIO.output(15,GPIO.HIGH)
    elif char == ord('b'): # stops the car (like a car brake, not a code break)
      b.ChangeDutyCycle(0)
      f.ChangeDutyCycle(0)

In [0]:
# restore curses and prepare for shutdown (important for later)
finally:
  curses.nocbreak(); screen.keypad(0); curses.echo()
#   echo may not be turned on if code from earlier is not altered
  curses.endwin()

In [0]:
# clean up GPIO pins for later
  GPIO.cleanup()

### Put all your code together and run it on remote desktop of PuTTY.

## CHEERS!

![alt text](https://drive.google.com/uc?id=14U5ae88hkDFiQQ180zMkSYBTfbZJnuM0)

### In the next update, I'll be attaching Ultrasonic Distance sensors and for object detection and avoidance, as well as a camera for video streaming and maybe a few other fun things!

## Products and useful links.

Starter pack for RPi 3 Model B+

![alt text](https://drive.google.com/uc?id=1XhA91JFZu5y7KHNW8_BNEc6swgoK9fKl)

### Products Used:

Cooling Fan:
https://www.amazon.com/gp/product/B07D3TVVC7/ref=ppx_yo_dt_b_asin_title_o04_s00?ie=UTF8&psc=1

On Off Switch:
https://www.amazon.com/gp/product/B01N2U8PK0/ref=ppx_yo_dt_b_asin_title_o09_s01?ie=UTF8&psc=1

L298N Motor Driver Controller:
https://www.amazon.com/gp/product/B00XAGRQTO/ref=ppx_yo_dt_b_asin_title_o02_s00?ie=UTF8&psc=1

CanaKit RPi 3 B+:
https://www.amazon.com/gp/product/B07BC567TW/ref=ppx_yo_dt_b_asin_title_o05_s02?ie=UTF8&psc=1

8 AA 12V Battery Case:
https://www.amazon.com/gp/product/B00VE7HBMS/ref=ppx_yo_dt_b_asin_title_o09_s00?ie=UTF8&psc=1

*Some pictures are taken directly from Amazon and/or reference links. Do not claim ownership.*

### Useful Links:

Raspberry gPIo (RPi.GPIO API): https://learn.sparkfun.com/tutorials/raspberry-gpio/python-rpigpio-api

Resistor for LED (Resistor Guide): http://www.resistorguide.com/resistor-for-led/

RPi.GPIO Basics: https://raspi.tv/2013/rpi-gpio-basics-5-setting-up-and-using-outputs-with-rpi-gpio

Here is some documentation on the L298N and PWM setup.

https://www.theengineeringprojects.com/2017/07/introduction-to-l298.html

https://learn.sparkfun.com/tutorials/raspberry-gpio/python-rpigpio-api


*2019-08-05: I utilized many resources throughout this build. I have tried to include all for reference. Additional resources may be added at a later date.*